In [1]:
import tensorflow

import pandas as pd
import numpy as np #데이터 배열화
import os #경로 설정용 
import keras #딥러닝용 패키지
import random #데이터 분산할 때 쓸 랜덤
import cv2 #이미지 읽기용
import math #연산용
import seaborn as sns #matplotlib에 다양한 시각화 기능이 추가된 패키지

from sklearn.metrics import confusion_matrix #분류의 정확성 평가
from sklearn.preprocessing import LabelBinarizer #데이터 전처리용
from sklearn.model_selection import train_test_split #데이터 분할용

import matplotlib.pyplot as plt #데이터 시각화용

from tensorflow.keras.layers import Dense,GlobalAveragePooling2D,Convolution2D,BatchNormalization #사용할 BN, ConV2 등의 계층
from tensorflow.keras.layers import Flatten,MaxPooling2D,Dropout #사용할 레이어

from tensorflow.keras.applications import DenseNet201
from tensorflow.keras.applications.densenet import preprocess_input #tensor나 numpy배열 전처리용

from tensorflow.keras.preprocessing import image #이미지 데이터를 실시간으로 처리하기 위한 도구
from tensorflow.keras.preprocessing.image import ImageDataGenerator,img_to_array #이미지 편집을 위한 제너레이터(Affine Transform)

from tensorflow.keras.models import Model #교육 및 추론 기능이 있는 개체로 레이어를 그룹화

from tensorflow.keras.optimizers import Adam #Adam 옵티마이저 사용. loss는 categorical_crossentropy 사용

#체크포인트를 두고 저장 + metric이 중지되면 학습률을 감소
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau 

import warnings #경고 무시
warnings.filterwarnings("ignore")

In [2]:
data=[]
labels=[]
random.seed(42)
imagePaths = sorted(list(os.listdir("emotion_FixedVer/")))
random.shuffle(imagePaths)
print(imagePaths)

for img in imagePaths:
    path=sorted(list(os.listdir("emotion_FixedVer/"+img)))
    for i in path:
        image = cv2.imread("emotion_FixedVer/"+img+'/'+i)
        image = cv2.resize(image, (48,48))
        image = img_to_array(image)
        data.append(image)
        l = label = img
        labels.append(l)

['neutral', 'fearful', 'happy', 'sad', 'angry', 'surprised']


In [3]:
data = np.array(data, dtype="float32") / 255.0
labels = np.array(labels)
mlb = LabelBinarizer()
labels = mlb.fit_transform(labels)
print(labels[0])

[0 0 0 1 0 0]


In [4]:
(xtrain,xtest,ytrain,ytest)=train_test_split(data,labels,test_size=0.2,random_state=42)
print(xtrain.shape, xtest.shape)

(22294, 48, 48, 3) (5574, 48, 48, 3)


In [5]:
datagen = ImageDataGenerator(zoom_range = 0.2, horizontal_flip=True, shear_range=0.2) #이미지 가공(학습률 향상을 위한 augmentation)
datagen.fit(xtrain)

In [6]:
model_d=DenseNet201(weights='imagenet',include_top=False, input_shape=(48, 48, 3)) #채널이 무조건 3개여야 하며 크기는 최소 32 이상

x=model_d.output

x=Flatten()(model_d.output)
x=Dense(512,activation='relu')(x)
x=Dropout(0.2)(x)
output=Dense(6,activation='softmax')(x) # because we have to predict the AUC
model=Model(model_d.input,output)


In [7]:
model.compile(
    optimizer = Adam(lr=0.1), 
    loss='categorical_crossentropy', 
    metrics=['accuracy']
  )

model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 48, 48, 3)]  0                                            
__________________________________________________________________________________________________
zero_padding2d (ZeroPadding2D)  (None, 54, 54, 3)    0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1/conv (Conv2D)             (None, 24, 24, 64)   9408        zero_padding2d[0][0]             
__________________________________________________________________________________________________
conv1/bn (BatchNormalization)   (None, 24, 24, 64)   256         conv1/conv[0][0]                 
______________________________________________________________________________________________

In [8]:
history = model.fit(datagen.flow(xtrain, ytrain, batch_size=32),
               epochs=100,
               validation_data=(xtest, ytest))

Epoch 1/100
697/697 [==============================] - 129s 154ms/step - loss: 3.8828 - accuracy: 0.2460 - val_loss: 1.7668 - val_accuracy: 0.2614
Epoch 2/100
697/697 [==============================] - 98s 141ms/step - loss: 1.7670 - accuracy: 0.2520 - val_loss: 1.7619 - val_accuracy: 0.2614
Epoch 3/100
697/697 [==============================] - 96s 138ms/step - loss: 1.7918 - accuracy: 0.2481 - val_loss: 1.7725 - val_accuracy: 0.2614
Epoch 4/100
697/697 [==============================] - 97s 139ms/step - loss: 1.7673 - accuracy: 0.2496 - val_loss: 1.7583 - val_accuracy: 0.2614
Epoch 5/100
697/697 [==============================] - 99s 141ms/step - loss: 1.7688 - accuracy: 0.2479 - val_loss: 1.7638 - val_accuracy: 0.2614
Epoch 6/100
697/697 [==============================] - 99s 142ms/step - loss: 1.7672 - accuracy: 0.2534 - val_loss: 1.7714 - val_accuracy: 0.1742
Epoch 7/100
697/697 [==============================] - 97s 140ms/step - loss: 1.7690 - accuracy: 0.2485 - val_loss: 1.7580 

Epoch 57/100
697/697 [==============================] - 96s 137ms/step - loss: 1.7666 - accuracy: 0.2482 - val_loss: 1.7584 - val_accuracy: 0.2614
Epoch 58/100
697/697 [==============================] - 96s 137ms/step - loss: 1.7679 - accuracy: 0.2479 - val_loss: 1.7602 - val_accuracy: 0.2614
Epoch 59/100
697/697 [==============================] - 96s 137ms/step - loss: 1.7666 - accuracy: 0.2515 - val_loss: 1.7725 - val_accuracy: 0.2614
Epoch 60/100
697/697 [==============================] - 97s 139ms/step - loss: 1.7660 - accuracy: 0.2526 - val_loss: 1.7544 - val_accuracy: 0.2614
Epoch 61/100
697/697 [==============================] - 97s 139ms/step - loss: 1.7676 - accuracy: 0.2515 - val_loss: 1.7555 - val_accuracy: 0.2614
Epoch 62/100
697/697 [==============================] - 97s 139ms/step - loss: 1.7669 - accuracy: 0.2512 - val_loss: 1.7573 - val_accuracy: 0.2614
Epoch 63/100
697/697 [==============================] - 95s 136ms/step - loss: 1.7679 - accuracy: 0.2488 - val_loss: 1

In [9]:
model.evaluate(xtest, ytest, batch_size = 32)

175/175 [==============================] - 8s 44ms/step - loss: 1.7717 - accuracy: 0.2614


[1.7716566324234009, 0.26139217615127563]

In [10]:
model.save('Densenet201.h5')

In [11]:
#완성도 확인
ypred = model.predict(xtest)

total = 0
accurate = 0
accurateindex = []
wrongindex = []

for i in range(len(ypred)):
    if np.argmax(ypred[i]) == np.argmax(ytest[i]): #가장 큰 값을 찾아 인덱스 반환
        accurate += 1
        accurateindex.append(i)
    else:
        wrongindex.append(i)
        
    total += 1
    
print('Total-test-data;', total, '\taccurately-predicted-data:', accurate, '\t wrongly-predicted-data: ', total - accurate)
print('Accuracy:', round(accurate/total*100, 3), '%')

Total-test-data; 5574 	accurately-predicted-data: 1457 	 wrongly-predicted-data:  4117
Accuracy: 26.139 %
